In [1]:
import numpy as np
import wandb
import copy
from tqdm import tqdm

In [2]:
api = wandb.Api()

In [7]:
summary_metrics_min = [ ]
summary_metrics_max = [
    "max_frac_segment50",
    "frac100_at_max_frac_segment50",
]
for f in [80, 85, 90, 93, 95]:
    summary_metrics_min.append(f"n_edges_frac_segment50_{f}")
    summary_metrics_max.append(f"efficiency_at_segment50_{f}")
    summary_metrics_max.append(f"frac100_at_segment50_{f}")
    summary_metrics_max.append(f"frac75_at_segment50_{f}")
    summary_metrics_max.append(f"purity_at_segment50_{f}")
summary_metrics_last = [
    "attractive",
    "repulsive",
    "attractive_train",
    "repulsive_train",
]

In [8]:
def update_summary_metrics(run):
    history = run.history()
    for key in summary_metrics_min:
        try:
            current = copy.copy(history[key])
            if isinstance(current, str):
                continue
            current = current.astype('float').to_numpy()
            if np.isfinite(current).any():
                new = np.nanmin(current).item()
                run.summary[key] = new
        except KeyError:
            pass
        except Exception as e:
            print(key, run)
            raise e
    for key in summary_metrics_max:
        try:
            current = copy.copy(history[key])
            if isinstance(current, str):
                continue
            current = current.astype('float').to_numpy()
            if np.isfinite(current).any():
                new = np.nanmax(current).item()
                run.summary[key] = new
        except KeyError:
            pass
        except Exception as e:
            print(key, run)
            raise e
    for key in summary_metrics_last:
        try:
            current = copy.copy(history[key])
            if isinstance(current, str):
                continue
            current = current.astype('float').to_numpy()
            if np.isfinite(current).any():
                new = current[np.isfinite(current)][-1].item()
                run.summary[key] = new
        except KeyError:
            pass
        except Exception as e:
            print(key, run)
            raise e
    run.summary.update()

In [ ]:
wandb.Run.finish()

In [9]:
runs = list(api.runs('gnn_tracking/gnn_tracking_gc', {
    "$and": [{
    'created_at': {
        "$gt": '2023-12-10'
        }
    }]
}))
print(f"Found {len(runs)} runs")

Found 59 runs


In [10]:
for run in tqdm(runs):
    update_summary_metrics(run)

100%|██████████| 59/59 [00:32<00:00,  1.81it/s]
